# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import pandas as pd
import numpy as np
import json
import csv

from pathlib import Path
from pprint import pprint

#### Creating list of filepaths to process original event csv data files

In [29]:
# Check path for filepath
pprint(str(Path(os.getcwd()).parent) + '/event_data')

'/Users/renan/projects/UDACITY/data-modeling-cassandra/event_data'


In [31]:
# Get your current folder and subfolder event data
filepath = str(Path(os.getcwd()).parent) + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     pprint(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
df = pd.read_csv('./event_datafile_new.csv')
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creating a Keyspace 
session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
"""
)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

---

## Create queries to ask the following three questions of the data

1. Give me the **artist**, **song title** and **song's length** in the music app history that was heard during **sessionId = 338**, and **itemInSession  = 4**;


2. Give me only the following: **name of artist**, **song** (sorted by **itemInSession**) and **user (first and last name)** for **userid = 10**, **sessionid = 182**;
    

3. Give me every **user name (first and last)** in my music app history **who listened to the song 'All Hands Against His Own'**.





## Query 1

### Give me:
- artist; 
- song title; and 
- song's length... 

...in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`.

In [9]:
# Utils -- run as needed

# session.execute("DROP TABLE music_library")

##### Creating `music_library` table:

In [10]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Common CREATE TABLE statement
create_if_not_exists = "CREATE TABLE IF NOT EXISTS {}"

# Creating table for music_library
create_music_libary_table = create_if_not_exists.format(
                                                        """
                                                            music_library ( \
                                                                    sessionId INT, \
                                                                    itemInSession INT, \
                                                                    artist_name TEXT, \
                                                                    song_title TEXT, \
                                                                    song_length FLOAT, \
                                                                    PRIMARY KEY(sessionId, itemInSession) \
                                                                ); \
                                                        """
                                                       )



# Executing query
session.execute(create_music_libary_table)

# print(create_music_libary_table)

##### Populating `music_library` table:

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## TO-DO: Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO music_library (sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
#         print(type(line[0]), type(line[9]), type(line[5]), type(float(line[8])), type(line[3]))
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

select_query = """
    SELECT artist_name, song_title, song_length FROM music_library WHERE sessionId = 338 and itemInSession = 4; 
"""

rows = session.execute(select_query)

for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

---

### Query 2

#### Give me only the following: 

- name of artist, 
- song (sorted by itemInSession) and 
- user (first and last name) 

for `userid = 10`, `sessionid = 182`.

In [13]:
# Utils -- uncomment to run as needed

# session.execute("DROP TABLE artist_song_user")

# for i, column in enumerate(df.columns):
#     print(i, column)

##### Creating `artist_song_user` table:

In [14]:
## TO-DO: Query 2: 
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Creating artist_song_user table
create_artist_song_user_table = create_if_not_exists.format(
                                                        """
                                                            artist_song_user ( \
                                                                    userId INT, \
                                                                    sessionId INT, \
                                                                    itemInSession INT, \
                                                                    artist TEXT, \
                                                                    firstName TEXT, \
                                                                    lastName TEXT, 
                                                                    song TEXT, \
                                                                    PRIMARY KEY((userId), sessionId, itemInSession) \
                                                                ); \
                                                        """
                                                       )



# Executing create_artist_song_user_table query
session.execute(create_artist_song_user_table)

##### Populating `artist_user_song` table:

In [15]:
# Populating table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## TO-DO: Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        # print(type(line[0]), type(line[9]), type(line[5]), type(float(line[8])), type(line[3]))
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

##### Creating and running query:

In [16]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

select_artist_song_user_query = """
    SELECT artist, firstName, lastName, song \
    FROM artist_song_user \
    WHERE userId = 10 AND sessionId = 182;
"""

rows_artist_song_user_query = session.execute(select_artist_song_user_query)

for row in rows_artist_song_user_query:
    print(row)

Row(artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


---

### Query 3

#### Give me only the following: 

- every **user name (first and last)** in my music app history who listened to the **song 'All Hands Against His Own'**

In [17]:
# Utils -- run as needed

# session.execute("DROP TABLE user_song_AHAHO")

# for i, column in enumerate(df.columns):
#     print(i, column)

##### Creating `user_song_AHAHO` table:

In [18]:
## TO-DO: Query 3: 

# Creating user_song_AHAHO table
create_user_song_AHAHO = create_if_not_exists.format(
                                                        """
                                                            user_song_AHAHO ( \
                                                                    userId INT, \
                                                                    firstName TEXT, \
                                                                    lastName TEXT, 
                                                                    song TEXT, \
                                                                    PRIMARY KEY(song) \
                                                                ); \
                                                        """
                                                       )



# Executing create_artist_song_user_table query
session.execute(create_user_song_AHAHO)

##### Populating `user_song_AHAHO` table:

In [19]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## TO-DO: Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO user_song_AHAHO (userId, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        # print(type(line[0]), type(line[9]), type(line[5]), type(float(line[8])), type(line[3]))
        
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

##### Writing and running query:

In [20]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

select_user_song_AHAHO = """
    SELECT firstName, lastName, song \
    FROM user_song_AHAHO \
    WHERE song = 'All Hands Against His Own';
"""

rows_artist_song_user_query = session.execute(select_user_song_AHAHO)

for row in rows_artist_song_user_query:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch', song='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions

tables = ['music_library', 'artist_song_user', 'user_song_AHAHO']

for table in tables:
    session.execute("DROP TABLE {}".format(table))

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()